
<a href="https://colab.research.google.com/github/isikus/MachineScorning/blob/f2148d889811422570bdad3ef77d4670fc9bac4e/Get%20Machine%20Scorning%20ruGPT3%20model%20output.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Get Machine Scorning ruGPT3 model output

In [1]:
import os

def is_empty(fpath):  
    return not (os.path.isfile(fpath) and os.path.getsize(fpath) > 0)

#@markdown First, let's choose a model:

model = "rukcna"  #@param ["wikihowto", "kish_verses", "products_desc", "rukcna"]

!wget -q --show-progress https://storage.googleapis.com/ml-bucket-isikus/machine-scorning/{model}.tar.gz
!mkdir {model}
!tar -xzf {model}.tar.gz -C {model}

!echo "Downloaded and extracted {model} model files"

rukcna.tar.gz       100%[===================>] 443.70M  54.2MB/s    in 9.3s    
Downloaded and extracted rukcna model files


Write your inputs to the model here below the `%%writefile input.txt` line:

In [2]:
%%writefile input.txt


Writing input.txt


If you are out of ideas, the following cell will fetch the example input instead in case you have not specified any input:

In [3]:
if is_empty("input.txt"):
  !cp {model}/example_input.txt input.txt

In [4]:
#@markdown Then, you should specify how much takes on each prompt should the model do (how many variants of outputs should we get for each line of input)

takes = 4  #@param {"type": "integer"}



## Install enviroment

Checking GPU card – the command below should work and show around 12-15k of available RAM

In [5]:
!nvidia-smi

Wed Dec 30 07:00:24 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.27.04    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8     7W /  75W |      0MiB /  7611MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [6]:
!pip3 install -q urllib3==1.25.4 transformers==2.8.0

!echo "Installed dependencies"

     |████████████████████████████████| 133kB 6.4MB/s 
     |████████████████████████████████| 573kB 8.5MB/s 
     |████████████████████████████████| 133kB 10.6MB/s 
     |████████████████████████████████| 3.7MB 13.3MB/s 
     |████████████████████████████████| 890kB 31.8MB/s 
     |████████████████████████████████| 1.1MB 45.5MB/s 
     |████████████████████████████████| 7.2MB 54.3MB/s 
     |████████████████████████████████| 71kB 9.9MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
Installed dependencies


In [7]:
!wget -q --show-progress https://raw.githubusercontent.com/sberbank-ai/ru-gpts/master/generate_transformers.py

!echo "Downloaded execution script"

generate_transforme 100%[===================>]  10.23K  --.-KB/s    in 0s      
Downloaded execution script


## Get results from the model

Now we finally generate batch results from our input. Here you should check the `add_randomness` flag to get different results: otherwise they'll be pretty similar (mostly like in the channel). Again, feel free to change `temperature`, `k` or `p` where appropriate to presumably get even more crappy results.

In [8]:
#@markdown ### Prepare model

import re
import os
import sys
import json
import random
from datetime import datetime

from IPython.display import Markdown, display

def printmd(string):
    display(Markdown(string))

with open(model+"/meta.json", "r") as injson:
  model_meta = json.load(injson)

# converting inputs

s = ""

with open("input.txt", "r", encoding="utf-8") as intxt:
  prompts = [p for p in intxt.read().split("\n") if p]

for prompt in prompts:
  s += (model_meta['prompt'] % prompt + '\n') * 4

with open("rawi.txt", "w", encoding="utf-8") as rawo:
  rawo.write(s)

In [9]:
%%time

#@markdown ### Start the generation!

add_randomness = False #@param {"type": "boolean"}

rs = random.choice(list(range(0, 10000))) if add_randomness else 42
print("Random seed:", rs)

responses = !python generate_transformers.py \
    --model_type=gpt2 \
    --model_name_or_path={model} \
    --k=5 \
    --p=0.95 \
    --seed={str(rs)} \
    --temperature=1.0 \
    --repetition_penalty=1.0 \
    --stop_token="{model_meta['sequence_end_character']}" \
    --length=500 < rawi.txt 2>/dev/null

responses = "\n".join(list(responses))

lbc = model_meta["line_break_character"]
ssc = model_meta["sequence_start_character"]
sec = model_meta["sequence_end_character"]
prompt = re.sub(r"%[a-z]+", "(.*?)", model_meta["prompt"])

found = re.findall(re.compile(r"Context >>> ruGPT:\n" + repr(prompt)[1:-1] + r"\n?(.*?)" + ssc, flags=re.DOTALL), responses)
for pt, rs in found:
  printmd('**' + pt + '**')
  printmd(re.sub(lbc, "<br>", rs).strip() + "\n")

Random seed: 42


**Путин пообещал рассмотреть вопрос об индексации пенсий работающим пенсионерам**

Премьер-министр кабинета министров КНДР Пак Бон Чжу по случаю 73-летия основания ТПК 7 октября направил поздравительную телеграмму премьер-министру кабинета министров Ли Су Ёну.<br>Он выразил уверенность в том, что отношения дружбы и сотрудничества между КНДР и Китаем, имеющие давнюю историю и традицию, будут расширяться и развиваться в соответствии с интересами народов двух стран и общими интересами народов двух стран. И сказал, что желает правительству и народу КНДР дальнейшего укрепления и развития отношений дружбы и сотрудничества между двумя странами.<br>-о-


**Путин пообещал рассмотреть вопрос об индексации пенсий работающим пенсионерам**

Премьер-министр России на заседании Госсовета КНДР 12-го числа встретился с президентом Мун Чжэ Ином, и имел беседу.<br>Президент сказал, что в нынешнем году, году 70-летия установления дипломатических отношений между КНДР и РФ, будет более укреплять и развивать двусторонние отношения. И отметил, что будет прилагать усилия к развитию двусторонних отношений, и отметил о том, что в нынешнем году будет более укреплять и развивать двусторонние отношения между КНДР и РФ.<br>Он подчеркнул, что желает корейскому народу больших успехов в борьбе за строительство экономической державы и процветание страны и отметил о том, что будет прилагать все усилия, чтобы в нынешнем году правительство КНДР внесло вклад в обеспечение стабильности на Корейском полуострове и в регионе.<br>Также он отметил, что будет прилагать усилия к укреплению самозащитных оборонных сил и укреплению сотрудничества с международными организациями в целях защиты мира Корейского полуострова и региона.<br>-о-


...

**В Казахстане обрушился мост через Вселенную**

В провинции Северной Пхёнъан провинции Северной Пхёнъан, где находится место катастрофы судна «Пуэбло-2», обрушились мост и площадь перед ним.<br>По сообщениям, в результате чего мост и площадь перед ним оказались затопленными.<br>В связи с этим в комитете провинции Канвон, центральном управлении гидрометеорологической службы и управлении по ликвидации последствий приняли меры по срочному восстановлению моста и площади перед ним.<br>-о-


**В Казахстане обрушился мост через Вселенную**

В последнее время на днях в разных местах Кореи, в том числе в провинции Канвон, в результате чего прервалось движение поездов на протяжении несколько сот км. и прервалось движение судов, в результате чего прервалось движение поездов на протяжении несколько сот км. и прервалось движение поездов на протяжении несколько сот км.


CPU times: user 1.25 s, sys: 211 ms, total: 1.46 s
Wall time: 10min 36s


## Get more results

Feel free to try another set of inputs by re-running the output below!

It is also strongly suggested to turn on `add_randomness` in order to get more diverse results

In [10]:
%%writefile input.txt


Overwriting input.txt


In [11]:
%%time

#@markdown Specify how much takes on each prompt should the model do (how many outputs should we get for each prompt)

takes = 4  #@param {"type": "integer"}

s = ""

with open("input.txt", "r", encoding="utf-8") as intxt:
  prompts = [p for p in intxt.read().split("\n") if p]

for prompt in prompts:
  s += (model_meta['prompt'] % prompt + '\n') * 4

with open("rawi.txt", "w", encoding="utf-8") as rawo:
  rawo.write(s)

#@markdown Choose if you want to add randomness (this is a must-check if you want some new results with the same prompts)

add_randomness = True #@param {"type": "boolean"}

rs = random.choice(list(range(0, 10000))) if add_randomness else 42
print("Random seed:", rs)

responses = !python generate_transformers.py \
    --model_type=gpt2 \
    --model_name_or_path={model} \
    --k=5 \
    --p=0.95 \
    --seed={str(rs)} \
    --temperature=1.0 \
    --repetition_penalty=1.0 \
    --stop_token="{model_meta['sequence_end_character']}" \
    --length=500 < rawi.txt 2>/dev/null

responses = "\n".join(list(responses))

lbc = model_meta["line_break_character"]
ssc = model_meta["sequence_start_character"]
sec = model_meta["sequence_end_character"]
prompt = re.sub(r"%[a-z]+", "(.*?)", model_meta["prompt"])

found = re.findall(re.compile(r"Context >>> ruGPT:\n" + repr(prompt)[1:-1] + r"\n?(.*?)" + ssc, flags=re.DOTALL), responses)
for pt, rs in found:
  printmd('**' + pt + '**')
  printmd(re.sub(lbc, "<br>", rs).strip() + "\n")

Random seed: 4193


**В России появится реестр сайтов, «причастных к нарушениям основополагающих прав и свобод человека»**

В России появится реестр ресурсов, «причастных к нарушениям основополагающих прав и свобод человека».<br>По словам заинтересованного лица реестр ресурсов, который находится на сайте Комитета по делам общественных наук Госдумы РФ и министерства образования и науки, является сайтом-справочником, где записаны все сайты, связанные с «незаконностью» ввоза и оборота оружия, наркотиков, взрывчатки, токсических веществ и их прекурсоров.<br>Этот реестр будет размещен на сайте комитета по делам общественных наук, который возглавляет первый секретарь комитета Госдумы по образованию Ли Су Ён.<br>На сайте размещены сайты, в том числе сайт «Грифон-персонал», сайт «Общество по изучению идей чучхе», сайт «Грифон-персонал», сайт по изучению идей чучхе и сайт по изучению идей чучхе и т.д.<br>По сайту будут помещены статьи, где подробно разъяснены содержания о том, что в России существуют сайты, где записаны содержания о «незаконности» ввоза и оборота оружия, наркотиков, взрывчатки, токсических веществ и их прекурсоров.<br>В реестр будут включены сайты, где записаны содержание «незаконности» ввоза и оборота оружия, наркотиков, взрывчатки, токсических веществ и их прекурсоров, и сайты, которые могут представлять материалы, связанные с незаконным оборотом наркотиков, взрывчатки, взрывчатки и т.д.<br>-о-


**В России появится реестр сайтов, «причастных к нарушениям основополагающих прав и свобод человека»**

По случаю 7-летия опубликования Совместной декларации от 15-го июня и Декларации от 4-го октября в России в Интернете-портале России «Патриоты Москва» размещен полный текст Декларации от 15-го июня, а в Интернете-портале - сайт «Патриоты Москва» России – полный текст декларации.<br>В Интернете-портале «Патриоты Москва» размещены фотоматериалы о заслугах великого вождя товарища Ким Ир Сена и великого руководителя товарища Ким Чен Ира в основании, укреплении и развитии Республики.<br>Также размещены труды великого Полководца и фотоматериалы, показывающие заслуги выдающихся исполинов в развитии межкорейских отношений.<br>На сайте «Патриоты Москва» размещен фотоматериал, показывающий заслуги выдающихся исполинов в строительстве социалистической державы.<br>В Интернете-портале «Патриоты Москва» размещена статья под заголовком «В поддержку исторической Пханмунчжомской декларации» и размещена статья под названием «Потерпев поражение на корейской войне, США и южнокорейские марионетки не могут скрыть своего поражения».<br>На сайте «Патриоты Москва» размещены труды товарища Ким Чен Ира и книги, представляющие действительность Кореи.<br>-о-


...

**Новым президентом России оказался Виктор Цой**

Председатель Президиума ВНС КНДР Ким Ен Нам 2 октября в Мансудэском дворце съездов вручил верительные грамоты президенту Российской Федерации Владимиру Владимировичу Путину.<br>В церемонии вручения приняли участие кадры правительства, в том числе зампред Президиума ВНС Чве Хви, председатель Президиума ВНС Ким Ен Нам, министр иностранных дел Ли Ён Хо, председатель Президиума ВНС Чве Бу Ир, соответствующие работники.<br>Также приняли участие деятели разных кругов России, включая председателя ВНС, председателя Совмина Республики Куба Мигеля Марио Диас-Канеля Бермудеса, председателя Российского комитета поддержки самостоятельного и мирного воссоединения Кореи Владимира Машмелина и председателя Российского общества дружбы и культурного сотрудничества с КНДР М. С. Баранникова.<br>Ким Ен Нам выступил с речью.<br>Президент Путин сказал, что рад тому, что отношения дружбы и сотрудничества между РФ и КНДР хорошо развиваются в соответствии с требованием новой эпохи, и выразил надежду и уверенность в том, что имеющие давнюю историю и традиции отношения дружбы и сотрудничества между двумя странами будут и впредь расширяться и развиваться в соответствии с общими интересами народов двух стран.<br>Президент Путин сказал, что будет прилагать активные усилия к развитию отношений между двумя странами, и выразил уверенность в том, что хорошие отношения дружбы и сотрудничества между РФ и КНДР будут более укрепляться и развиваться в соответствии со стремлениями и чаяниями народов двух стран.<br>Президент Путин сказал, что Россия придает большое значение отношениям дружбы и сотрудничества между двумя странами, и выразил благодарность за это.<br>Президент Путин выразил уверенность в том, что отношения дружбы и сотрудничества между двумя странами будут и впредь расширяться и развиваться в соответствии с интересами народов двух стран.<br>Президент Путин подчеркнул, что будет прилагать усилия к укреплению и развитию отношений дружбы и сотрудничества между двумя странами в разных сферах, и отметил, что будет прилагать совместные усилия к дальнейшему укреплению и развитию российско-корейских отношений дружбы и сотрудничества в разных сферах.<br>-о-


**Новым президентом России оказался Виктор Цой**

В связи с тем, что в последние дни врио президента России господин Ким Чен Ын занял должность президента России, 2-го числа опубликован указ Президиума Верховного Народного Собрания КНДР.<br>В указе отмечено, что новый президент России Владимир Путин является президентом, пользующимся уважением международного сообщества, и что он будет прилагать активные усилия для укрепления и развития отношений дружбы и сотрудничества между двумя странами.<br>Указ определил срок полномочий президента – 5 лет с момента выдвижения на пост президента.<br>Также определено, что в период с 2-го октября по 2-ое ноября назначить новым президентом России Ким Ен Намом, председателем Корейского общества по изучению идей чучхе, председателем Общества дружбы и культурного сотрудничества с КНДР, министром иностранных дел Российской Федерации Сергеем Лавровым и председателем Совета Федерации России Валентиной Ивановной Матвиенко, председателем комитета по федеративному устройству, региональной политике, местному самоуправлению и проблемам Севера и Юга Совета Федерации, главой комитета по федеративному устройству, региональной политике, местному самоуправлению и делам Севера Госдумы России.<br>Также указ определил с 2-го ноября по 17-ое декабря периодом для выражения позиции России по отношению к КНДР, и объявил о ниже следующем.<br>1. Будем прилагать активные усилия к укреплению и развитию отношений дружбы и сотрудничества между двумя странами в соответствии с требованием новой эпохи и нового этапа развития.<br>2. Будем активно прилагать усилия к развитию двусторонних отношений.<br>3. Будем укреплять и развивать отношения дружбы и сотрудничества с КНДР.<br>-о-


CPU times: user 538 ms, sys: 109 ms, total: 647 ms
Wall time: 4min 6s


Have fun (hopefully)!